# Stacked Machine Learning

In [1]:
## IMPORTANT !

# In the first order need to set the number of CPU 
# for calculation before launching (depends on computer's number of cores)
n_jobs= 40

### Load libraries

In [2]:
#libraries
import pandas as pd
import numpy as np
import os
import sys
import shutil
import glob
import joblib
import warnings
from datetime import date, datetime

from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import LeavePGroupsOut
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import pearsonr
import scipy.stats as st

from nilearn import image as nli
from nilearn import plotting

#from mne.viz import plot_connectivity_circle
from mne_connectivity.viz import plot_connectivity_circle


from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning


from scipy.stats import zscore as  zscore
import pickle

### Load functions

In [3]:
def control_2(z, control, index):    #age+gender+race/ethn == 4
    #z should be a series
    #control is a feature table
    #index for indexing
    
    #shrink data to local train index
    y = z.reindex(index = index)
    X = control.reindex(index = index)

    ind_y = np.array(y.index)
    
    #Centralize target by y_i-y_mean
    y= pd.DataFrame([i-y.mean() for i in y], index=y.index)    
    #y_real = y
    
    #reshaping data
    if len(X.values.shape) == 1:
        X = X.values.reshape(-1, 1)
    else:
        X = X.values
    y = y.values.reshape(-1, 1).ravel()
    
    #fill Nan in X
    X = SimpleImputer(strategy='mean').fit_transform(X)
    
    #Standartize X
    X = StandardScaler().fit_transform(X)
    
    #Fit to the training set
    y_pred = LinearRegression().fit(X, y).predict(X)
    
    y_res = y - y_pred
    
    return y_res, ind_y

In [4]:
def elnet(X, y):

    #drop Nan in target and clean this subj from features
    y = y.dropna()
    X = X.reindex(index=y.index)
    ind_y = np.array(y.index)
      
    y_real=y
    
    #reshaping data
    X = X.values
    y = y.values.reshape(-1, 1).ravel()
    
    #Standartize X
    X = StandardScaler().fit_transform(X)
    
    # Setup the pipeline steps:
    steps = [('elasticnet', ElasticNet(random_state=42))]

    # Create the pipeline: pipeline 
    pipeline = Pipeline(steps)

    # Specify the hyperparameter space
    parameters = {'elasticnet__alpha': np.logspace(-1, 2, 70),
                  'elasticnet__l1_ratio':np.linspace(0,1,25)}

    # Create the GridSearchCV object:
    gm_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=n_jobs)
    
    # Fit to the training set
    gm_cv.fit(X, y)
    
    #predict new y
    y_pred = gm_cv.predict(X)

    # Compute and print the metrics
    acc = gm_cv.best_score_
    bpar = gm_cv.best_params_
    model = gm_cv.best_estimator_
    mse = mean_squared_error(y_real, y_pred)
    mae = mean_absolute_error(y_real, y_pred)
    corr, _ = pearsonr(np.array(y_real.values.reshape(-1, 1).ravel(), dtype=float), np.array(y_pred, dtype=float))
            
    return bpar['elasticnet__alpha'], bpar['elasticnet__l1_ratio'], acc, mse, corr, model, y_pred, mae

In [5]:
def reaply_ElNet(X, y, model):
    # param should be pd.Series with indexes from model
    
    #drop Nan in target and clean this subj from features
    y = y.dropna()
    X = X.reindex(index =y.index)
    ind_y = np.array(y.index)  # indexes as separate variable 
    
    y_real = y

    #reshaping data
    X = X.values
    y = y.values.reshape(-1, 1).ravel()
    
    #Standartize X
    X = StandardScaler().fit_transform(X)
    
    #predict new y
    y_pred = model.predict(X)
    
    # Compute and print the metrics
    bacc = model.score(X, y)
    mse = mean_squared_error(y_real, y_pred)
    mae = mean_absolute_error(y_real, y_pred) 
    corr, _ = pearsonr(np.array(y_real.values.reshape(-1, 1).ravel(), dtype=float), np.array(y_pred, dtype=float))
    
    return y_pred, y_real, ind_y, bacc, mse, corr, mae

### Path to the tables folder

In [6]:
path='/media/hcs-psy-narun/Alina/Lifespan_Projects_NonTask_Comparison/'

### Load tables

#### Load HCP YA

In [26]:
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

#targets table
targ_YA = pd.read_csv(path+'HCP_YA/'+'cognition_table.csv', index_col=0)['CogTotalComp_Unadj'].dropna()

#demography
demo_YA = pd.read_csv(path+'HCP_YA/'+'demographics_table.csv', index_col=0)
demo_YA = demo_YA.reindex(index=targ_YA.index) #filter subjects to target subj length

#features tables as dictionary
features_YA = {
    'cort':pd.read_csv(path+'HCP_YA/'+'cort_table.csv', index_col=0),
    'subc':pd.read_csv(path+'HCP_YA/'+'subc_table.csv', index_col=0),
    'surf':pd.read_csv(path+'HCP_YA/'+'surf_table.csv', index_col=0),
    'volBrain':pd.read_csv(path+'HCP_YA/'+'VolBrain_table.csv', index_col=0),
    'rest':pd.read_csv(path+'HCP_YA/'+'Rest_FC_group_z_full.csv', index_col=0)}

#filter to target subject number
for key in features_YA.keys():
    features_YA[key] = features_YA[key].reindex(index=targ_YA.index)

#rename columns for cort and area
for mod in ['cort', 'surf']:
    features_YA[mod].columns = [mod+'_'+i for i in features_YA[mod].columns]

#create tables with 2 controling parameters: age, sex
sex_coded = pd.Series(LabelEncoder().fit_transform(demo_YA.loc[:,['Gender']]), index=demo_YA.index, name='Gender')

control_YA = pd.concat([sex_coded, demo_YA.loc[:, ['Age_in_Yrs']]], axis=1) #

#### Load HCP A

In [27]:
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

#targets table
targ_A = pd.read_csv(path+'HCP_A/'+'cognition.csv', index_col=0)['nih_totalcogcomp_unadjusted'].dropna()

#demography
demo_A = pd.read_csv(path+'HCP_A/'+'demography.csv', index_col=0)
demo_A = demo_A.reindex(index=targ_A.index) #filter subjects to target subj length

#features tables as dictionary
features_A = {
    'cort':pd.read_csv(path+'HCP_A/'+'cort.csv', index_col=0),
    'subc':pd.read_csv(path+'HCP_A/'+'subc.csv', index_col=0),
    'surf':pd.read_csv(path+'HCP_A/'+'surf.csv', index_col=0),
    'volBrain':pd.read_csv(path+'HCP_A/'+'VolBrain.csv', index_col=0),
    'rest':pd.read_csv(path+'HCP_A/'+'rest_hpass.csv', index_col=0)}

#filter to target subject number
for key in features_A.keys():
    features_A[key] = features_A[key].reindex(index=targ_A.index)

#rename columns for cort and area
for mod in ['cort', 'surf']:
    features_A[mod].columns = [mod+'_'+i for i in features_A[mod].columns]

#create tables with 2 controling parameters: age, sex
sex_coded = pd.Series(LabelEncoder().fit_transform(demo_A.loc[:,['sex']]), index=demo_A.index, name='sex')

control_A = pd.concat([sex_coded, (demo_A.loc[:, ['interview_age']]) ], axis=1) #

#### Load DUD

In [28]:
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

#targets table
targ_D = pd.read_csv(path+'DUD_45/'+'info.csv', index_col=0)['fsiq45'].dropna()

#demography
demo_D = pd.read_csv(path+'DUD_45/'+'info.csv', index_col=0)['sex']
demo_D = demo_D.reindex(index=targ_D.index) #filter subjects to target subj length

#features tables as dictionary
features_D = {
    'cort':pd.read_csv(path+'DUD_45/'+'cort_thck.csv', index_col=0),
    'subc':pd.read_csv(path+'DUD_45/'+'subc_vol.csv', index_col=0),
    'surf':pd.read_csv(path+'DUD_45/'+'cort_area.csv', index_col=0),
    'volBrain':pd.read_csv(path+'DUD_45/'+'total_vol.csv', index_col=0),
    'rest':pd.read_csv(path+'DUD_45/'+'rest.csv', index_col=0)}

#filter to target subject number
for key in features_D.keys():
    features_D[key] = features_D[key].reindex(index=targ_D.index)

#rename columns for cort and area
for mod in ['cort', 'surf']:
    features_D[mod].columns = [mod+'_'+i for i in features_D[mod].columns]

#create tables with 2 controling parameters: age, sex
sex_coded = pd.Series(LabelEncoder().fit_transform(demo_D), index=demo_D.index, name='sex')

control_D = sex_coded

In [ ]:
#make the same labels for columns in HCP YA and HCP A
features_YA['rest'].columns = features_A['rest'].columns

##### Adjust the brain features (age, sex, etc, depends on control table)

In [30]:
#control modalities 

Features = {}
for features, targ, control, dset in zip([features_YA, features_A, features_D], 
                                         [targ_YA, targ_A, targ_D], 
                                         [control_YA, control_A, control_D], 
                                         ['YA', 'A', 'D']):
    Features[dset] = {}
    Features[dset]['features_res'] = {}
    Features[dset]['targ'] = {}
    Features[dset]['model_PCA'] = {}
    
    #control target
    p1, p2 = control_2(targ, control, targ.index)
    targ = zscore(pd.Series(p1, index = p2))      #zscore

    #control modalities
    features_res = {}
    for key in features.keys():
        print('controlling ', key, datetime.now())

        #controlling the remaining for 2 parameters (age+sex)
        if key in ['cort', 'surf', 'subc', 'volBrain',  'rest']:
            d = {}
            for col in features[key].columns:
                p1,p2 = control_2(features[key][col], control, targ.index)
                d[col] = p1
            df= pd.DataFrame(d, index = p2)
            features_res[key] = zscore(df)   #zscore


    #keep rest residuals as a separate var
    res_rest = features_res['rest']
    res_rest_st = pd.DataFrame((StandardScaler().fit_transform(res_rest)), 
                                  index=res_rest.index, columns=res_rest.columns)

    #apply PCA to resting state
    pca = PCA(n_components=75, random_state=11)
    pca.fit(res_rest_st.values)
    rest_pca1 = pd.DataFrame(pca.transform(res_rest_st.values), index=res_rest.index)

    #assign pca table to rest features
    features_res['rest_PCA'] = rest_pca1
    

    
    Features[dset]['features_res'] = features_res
    Features[dset]['targ'] = targ
    Features[dset]['model_PCA'] = pca
    
    

controlling  cort 2023-09-29 23:12:51.939739
controlling  subc 2023-09-29 23:12:54.466511
controlling  surf 2023-09-29 23:12:54.795880
controlling  volBrain 2023-09-29 23:12:57.302943
controlling  rest 2023-09-29 23:12:57.394094
controlling  cort 2023-09-29 23:33:25.427101
controlling  subc 2023-09-29 23:33:27.012377
controlling  surf 2023-09-29 23:33:27.214079
controlling  volBrain 2023-09-29 23:33:28.777552
controlling  rest 2023-09-29 23:33:28.834133
controlling  cort 2023-09-29 23:45:51.053109
controlling  subc 2023-09-29 23:45:53.356113
controlling  surf 2023-09-29 23:45:53.656907
controlling  volBrain 2023-09-29 23:45:56.059686
controlling  rest 2023-09-29 23:45:56.139268


In [31]:
#check numbers
for tg in [targ_YA, targ_A, targ_D]:
    print(len(tg))

873
504
754


### ML script

In [35]:
os.mkdir(path+'output_3dsets_stack_newREST')
path_out = path+'output_3dsets_stack_newREST/'

In [36]:
os.mkdir(path_out+'PCA_models')

In [37]:
for key in Features.keys():
    joblib.dump(Features[key]['model_PCA'], (path_out+'PCA_models'+'/rest_'+key+'_pca_model.sav'))

In [38]:
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = "ignore"


i=0

Outputs = {}

for KEY in list(Features.keys()):
    print(KEY)    
    oth = list(Features.keys())
    oth.remove(KEY)
    
    Outputs[KEY]={}
    
    #assign train and test
    Train = Features[KEY]
    Test1 = Features[oth[0]]
    Test2 = Features[oth[1]]
    
    
    ###### part 1: TRAIN 
    
    Outputs[KEY]['Train']={}

    
    #Launch ElasticNet
    dict_perf={}        #to store modality performans
    dict_elnet_model={}  #to store enet model
    dict_ypred={}       #to store predicted values

    for key in Train['features_res'].keys():
        
        if key != 'rest':
            
            print('start ', str(key), datetime.now())   #print start time of calculations
            
            bpar1, bpar2, acc, mse, corr, model, y_pred, mae = elnet(Train['features_res'][key], 
                                                                     Train['targ']) #ML
        
            dict_perf[key] = acc, mse, mae, corr, bpar1, bpar2 
            dict_elnet_model[key] = model
            dict_ypred[key] = y_pred
    
    #run separatelly for stack
    
    print('start ', 'stack', datetime.now())   #print start time of calculations
    
    #table with predicted values
    stack_tab = pd.DataFrame(dict_ypred, index=Train['targ'].index)
    
    stack_STD_model = StandardScaler().fit(stack_tab.values)
    
    stack_tab_std = pd.DataFrame(stack_STD_model.transform(stack_tab.values), 
                                 index=stack_tab.index, columns=stack_tab.columns)
            
    bpar1, bpar2, acc, mse, corr, model, y_pred, mae = elnet(stack_tab_std, Train['targ']) #ML
        
    dict_perf['stack'] = acc, mse, mae, corr, bpar1, bpar2 
    dict_elnet_model['stack'] = model
    dict_ypred['stack'] = y_pred
    
        
    df_perf = pd.DataFrame(dict_perf, index=['best score r2', 'mse', 'mae','corr', 'best alpha', 'best l1_ratio'])
    df_y_pred = pd.DataFrame(dict_ypred, index=Train['targ'].index) 
    df_y_pred['origin_y'] = Train['targ']
    
    Outputs[KEY]['Train']['performance'] = df_perf
    Outputs[KEY]['Train']['y_pred'] = df_y_pred
    Outputs[KEY]['Train']['models'] = dict_elnet_model
    Outputs[KEY]['Train']['stack_tab'] = stack_tab
    Outputs[KEY]['Train']['stack_tab_std'] = stack_tab_std
    Outputs[KEY]['Train']['stack_STD_model'] = stack_STD_model
    
    ###### part 2: TEST
    
    j=1
    for Test in [Test1, Test2]:
        
        print('start ', 'Test'+'_'+str(j), datetime.now()) 
        
        Outputs[KEY]['Test'+'_'+str(j)]={}
        
        #apply PCA from Train
        
        #apply PCA to resting state
        rest_pca = pd.DataFrame(Train['model_PCA'].transform(Test['features_res']['rest'].values), 
                                index=Test['features_res']['rest'].index)

        
        
        #apply trained Enet models
        
        dict_perf={}        #to store modality performans
        dict_ypred={}
        
        for key in list(Test['features_res'].keys()):
            
            if key!='rest' and key!='rest_PCA':
                                           
                y_pred, y_real, ind_y, bacc, mse, corr, mae = reaply_ElNet(Test['features_res'][key], 
                                                                           Test['targ'], 
                                                                           Outputs[KEY]['Train']['models'][key]) #ML
                dict_ypred[key] = y_pred
                dict_perf[key] = bacc, mse, mae, corr
                
        #run separatelly for test pca and stack
        for key in ['rest_PCA', 'stack']:
            
            if key == 'rest_PCA':
                
                feat = rest_pca
                
                y_pred, y_real, ind_y, bacc, mse, corr, mae = reaply_ElNet(feat,
                                                                           Test['targ'],
                                                                           Outputs[KEY]['Train']['models'][key]) #ML
                
                dict_ypred[key] = y_pred
                dict_perf[key] = bacc, mse, mae, corr
            
            else:
                
                #table with pred-values
                stack_test = pd.DataFrame(dict_ypred, index = Test['targ'].index)
                
                stack_test_std = pd.DataFrame(stack_STD_model.transform(stack_test.values),
                                              index=stack_test.index, columns=stack_test.columns)
                
                y_pred, y_real, ind_y, bacc, mse, corr, mae = reaply_ElNet(stack_test_std,
                                                                           Test['targ'],
                                                                           Outputs[KEY]['Train']['models'][key]) #ML
                
                dict_ypred[key] = y_pred
                dict_perf[key] = bacc, mse, mae, corr

        
        

        df_perf = pd.DataFrame(dict_perf, index=['best score r2', 'mse', 'mae','corr'])
        df_ypred = pd.DataFrame(dict_ypred, index=Test['targ'].index)
        df_ypred['origin_y'] = Test['targ']
        
        Outputs[KEY]['Test'+'_'+str(j)]['performance'] = df_perf
        Outputs[KEY]['Test'+'_'+str(j)]['y_pred'] = df_ypred
        Outputs[KEY]['Test'+'_'+str(j)]['dset'] = oth[j-1]
        Outputs[KEY]['Test'+'_'+str(j)]['stack_tab'] = stack_test
        Outputs[KEY]['Test'+'_'+str(j)]['stack_test_std'] = stack_test_std
        Outputs[KEY]['Test'+'_'+str(j)]['rest_PCA'] = rest_pca
        
        
        j+=1
    
    
    
    
    i+=1

YA
start  cort 2023-09-30 00:03:51.782071
start  subc 2023-09-30 00:04:05.084574
start  surf 2023-09-30 00:04:14.101919
start  volBrain 2023-09-30 00:04:24.198526
start  rest_PCA 2023-09-30 00:04:32.722565
start  stack 2023-09-30 00:04:41.926542
start  Test_1 2023-09-30 00:04:51.148789
start  Test_2 2023-09-30 00:04:51.278026
A
start  cort 2023-09-30 00:04:51.461976
start  subc 2023-09-30 00:05:00.748794
start  surf 2023-09-30 00:05:09.413521
start  volBrain 2023-09-30 00:05:18.819790
start  rest_PCA 2023-09-30 00:05:29.345604
start  stack 2023-09-30 00:05:38.309323
start  Test_1 2023-09-30 00:05:46.345012
start  Test_2 2023-09-30 00:05:46.535314
D
start  cort 2023-09-30 00:05:46.718837
start  subc 2023-09-30 00:05:57.854155
start  surf 2023-09-30 00:06:06.539581
start  volBrain 2023-09-30 00:06:22.953347
start  rest_PCA 2023-09-30 00:06:31.553851
start  stack 2023-09-30 00:06:40.569015
start  Test_1 2023-09-30 00:06:48.899783
start  Test_2 2023-09-30 00:06:49.092783


In [39]:
### save features and  results

import pickle

with open(path_out+'Features.pkl', 'wb') as handle:
    pickle.dump(Features, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(path_out+'Outputs.pkl', 'wb') as handle:
    pickle.dump(Outputs, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [40]:
### show prediction accuracy indexes

for KEY in list(Outputs.keys()):
    print('Dataset ', KEY, ' as TRAIN')
    #print('Train')
    #display(Outputs[KEY]['Train']['performance'].iloc[:-2,:])
    print(' ')
    
    print('Test 1, Dataset ', Outputs[KEY]['Test_1']['dset'])
    display(Outputs[KEY]['Test_1']['performance'])
    print(' ')
    
    print('Test 2, Dataset ', Outputs[KEY]['Test_2']['dset'])
    display(Outputs[KEY]['Test_2']['performance'])
    print(' ')
    print(' ')

Dataset  YA  as TRAIN
 
Test 1, Dataset  A


,cort,subc,surf,volBrain,rest_PCA,stack
best score r2,0.043808,0.069221,0.031504,0.042935,0.080683,0.117003
mse,0.956192,0.930779,0.968496,0.957065,0.919317,0.882997
mae,0.787604,0.778418,0.796823,0.784424,0.765129,0.752249
corr,0.228624,0.286252,0.191306,0.207227,0.284121,0.358577


 
Test 2, Dataset  D


,cort,subc,surf,volBrain,rest_PCA,stack
best score r2,-0.009049,0.047533,0.023122,0.075002,0.047376,0.039155
mse,1.009049,0.952467,0.976878,0.924998,0.952624,0.960845
mae,0.805389,0.779181,0.787158,0.762989,0.774022,0.774610
corr,0.028215,0.226428,0.168746,0.284097,0.221023,0.253031


 
 
Dataset  A  as TRAIN
 
Test 1, Dataset  YA


...

,cort,subc,surf,volBrain,rest_PCA,stack
best score r2,-0.029258,0.042802,0.001720,0.082447,-0.006140,-0.118778
mse,1.029258,0.957198,0.998280,0.917553,1.006140,1.118778
mae,0.811239,0.779492,0.793407,0.760077,0.800571,0.846465
corr,0.085084,0.207294,0.111171,0.306955,0.172207,0.195772


 
 
Dataset  D  as TRAIN
 
Test 1, Dataset  YA


,cort,subc,surf,volBrain,rest_PCA,stack
best score r2,-0.025379,0.018766,0.024264,0.035200,0.051982,-0.016920
mse,1.025379,0.981234,0.975736,0.964800,0.948018,1.016920
mae,0.822247,0.806483,0.802232,0.801480,0.783472,0.799807
corr,0.036593,0.160045,0.178881,0.201263,0.233578,0.236469


 
Test 2, Dataset  A


,cort,subc,surf,volBrain,rest_PCA,stack
best score r2,-0.004504,0.033447,-0.020795,0.038386,-0.009307,-0.074229
mse,1.004504,0.966553,1.020795,0.961614,1.009307,1.074229
mae,0.809268,0.789476,0.815155,0.788838,0.801066,0.828596
corr,0.091103,0.192780,0.103317,0.206388,0.122358,0.166632


In [ ]:
# Load data (deserialize)
#with open('filename.pickle', 'rb') as handle:
#    unserialized_data = pickle.load(handle)